In [14]:
import pandas as pd
import os

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

from fuzzywuzzy import process

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch

import io
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

In [15]:
# Cargar el archivo Excel
file_path = '/Users/floroppen/Desktop/Python_projects/Scripts/Vendemos.xlsx'
df = pd.read_excel(file_path, header=1) 
df.head()

,Producto,Entrega,Precio Venta (USD),% de descuento aplicado al precio nuevo o similar,precio nuevo o similar (dolar) c/u,Disponibilidad,Interes,Cantidad,Observaciones,Link nueva / equivalente (no son mias estas publicaciones),Fotos reales
0,Asus zenpad Z10,Inmediata,59.5,0.30,85.0,Vendido,NaN,1,puede agregarse un chip para 4G. tiene 32gb de...,https://www.ebay.com/itm/225812413452?epid=238...,https://drive.google.com/drive/folders/1DnPYwl...
1,Auriculares Razer Kraken Kitty,Inmediata,168,0.30,240.0,Disponible,2.0,1,Auriculares Razer Kraken Kitty con luz rgb LED...,https://www.mercadolibre.com.ar/auriculares-ga...,https://drive.google.com/drive/folders/1reT_o2...
2,Batidora Mondial,Inmediata,30,0.30,44.0,Disponible,3.0,1,"Batidora Mondial Bella Massa Inox, 400w de pot...",https://www.mercadolibre.com.ar/batidora-de-pi...,https://drive.google.com/drive/folders/1iyIVWn...
3,Bicicleta fija,Inmediata,130,0.60,333.0,Vendido,NaN,1,"Tiene algunas marcas por maravillosos felinos,...",https://articulo.mercadolibre.com.ar/MLA-60235...,https://drive.google.com/drive/folders/1wHpQb4...
4,Bleff,Inmediata,14.1,0.25,18.8,Vendido,NaN,1,"Juego de Mesa Bleff, cerrado",https://www.mercadolibre.com.ar/juego-de-mesa-...,https://drive.google.com/drive/folders/1wLJDZh...


In [16]:
df = df[['Producto','Disponibilidad', 'Precio Venta (USD)', 'Observaciones', 'Fotos reales', 'Interes']]

In [17]:
df.shape

(53, 6)

In [18]:
df['Disponibilidad']

0       Vendido 
1     Disponible
2     Disponible
3        Vendido
4        Vendido
5        Vendido
6     Disponible
7     Disponible
8     Disponible
9        Vendido
10    Disponible
11    Disponible
12    Disponible
13       Vendido
14    Disponible
15    Disponible
16    Disponible
17       Vendido
18       Vendido
19    Disponible
20    Disponible
21       Vendido
22       Vendido
23       Vendido
24       Vendido
25    Disponible
26       Vendido
27    Disponible
28    Disponible
29    Disponible
30    Disponible
31     Reservado
32       Vendido
33      Vendido 
34     Reservado
35       Vendido
36    Disponible
37       Vendido
38    Disponible
39    Disponible
40    Disponible
41    Disponible
42       Vendido
43    Disponible
44    Disponible
45    Disponible
46    Disponible
47    Disponible
48    Disponible
49     Reservado
50       Vendido
51       Vendido
52       Vendido
Name: Disponibilidad, dtype: object

In [19]:
df2 = df[(df['Disponibilidad'] == 'Disponible')].copy()

df2['Folder ID'] = df2['Fotos reales'].apply(lambda x: x.split('/')[-1])
df2.head(20)

,Producto,Disponibilidad,Precio Venta (USD),Observaciones,Fotos reales,Interes,Folder ID
1,Auriculares Razer Kraken Kitty,Disponible,168,Auriculares Razer Kraken Kitty con luz rgb LED...,https://drive.google.com/drive/folders/1reT_o2...,2.0,1reT_o2GguU4Rwr4ZJ5IiEubJz26OrtHz?usp=sharing
2,Batidora Mondial,Disponible,30,"Batidora Mondial Bella Massa Inox, 400w de pot...",https://drive.google.com/drive/folders/1iyIVWn...,3.0,1iyIVWn9xKFSsKe8rQ-F1Ep7KSsB-JY2t
6,Calentador de agua,Disponible,60,"Calentador de agua sumergible, para 5 galones,...",https://drive.google.com/drive/folders/1KLC2sP...,18.0,1KLC2sPBFvuMr6lzRoNn8ja4C1-PmvuOt
7,Camara profesional Canon 80D,Disponible,Precio Venta (USD),La compré usada y no la usé casi nada al final...,https://drive.google.com/drive/folders/18_zCfz...,3.0,18_zCfz5gQADgjH3Qsz9qsJM-H4rC8XEf
8,Lente camara canon 10-18mm,Disponible,165,"Canon EF-S 10-18mm f/4.5-5.6 is STM Lens, prim...",https://drive.google.com/drive/folders/16o64rv...,3.0,16o64rvttQ4SiZ36Mu8_huwBcZ9nocVmt
10,Cargador portatil APC M10,Disponible,36,Cargador portatil APC M10,https://drive.google.com/drive/folders/1-8q1VV...,5.0,1-8q1VVyEv8TUxalRUO9wo7fR8oH4KGHY?usp=drive_link
11,Casio fx-95ES PLUS,Disponible,22,"Calculadora científica, bastante uso pero impe...",https://drive.google.com/drive/folders/1hnjdXm...,3.0,1hnjdXml6iwAKXpOGDT-_McSthXzqiC8N
12,Casio fx-991ES,Disponible,31,"Calculadora científica, casi sin uso",https://drive.google.com/drive/folders/1_PSi-N...,3.0,1_PSi-NaAAvZO2v6xDaDDjBSSkNSTJiv0
14,Cuadro de juegos de fromsoftware,Disponible,14,Canvas de juegos de Fromsoftware,https://drive.google.com/drive/folders/1LoBPvh...,4.0,1LoBPvhsRRKtJJqP6DEbnpICEvucZVLAi
15,Cuadro de majoras mask,Disponible,14,Canvas Majoras Mask,https://drive.google.com/drive/folders/1zmKzNQ...,4.0,1zmKzNQRrFEhTGTWXL75aWfQNUx6232iK


In [20]:
df2.shape

(29, 7)

In [21]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request

# Si el token.json existe, usa ese para crear las credenciales, si no, genera uno nuevo
def create_service():
    SCOPES = ['https://www.googleapis.com/auth/drive']
    creds = None
    # El archivo token.json guarda el acceso y actualización de tokens del usuario, y es
    # creado automáticamente cuando el flujo de autorización se completa por primera vez.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # Si no hay credenciales válidas disponibles, deja que el usuario se loguee.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secrets.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Guarda las credenciales para la próxima ejecución
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('drive', 'v3', credentials=creds)
    return service

service = create_service()

In [22]:
#Como el nombre de las cosas en df2 y las carpetas en Drive no son identicas, primero voy a listar el nombre de las carpetas y sus ids

def obtener_nombres_ids_carpetas(folder_id):
    """Obtiene los nombres e IDs de las carpetas dentro del folder_id proporcionado usando googleapiclient."""
    carpetas = []
    query = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false"
    response = service.files().list(q=query, fields='files(id, name)').execute()
    
    for file in response.get('files', []):
        carpetas.append((file['name'], file['id']))  # Using 'name' and 'id' from the response
    
    return carpetas

# Use the real ID of your main folder
folder_id_principal = '1FZNsil8joasf0Yp7ccthhDWTAl8pkcnr'
carpetas = obtener_nombres_ids_carpetas(folder_id_principal)

print(carpetas)

[('MONITOR_GAMER_SAMSUNG', '1BIul4-_RUq4cH6Ocelii5_4Hh3SqxX5F'), ('TACHOS_CHICOS', '1-sulxkenpq0D1wz7fSyB_jBRm5Ej8kF9'), ('TELE_SAMSUNG_4K', '1mo8vcZEpgryln75KE26F7Xde4ZWBrlnR'), ('ROLLERS', '1D16o3zeYynoEOi0gdWRzwZ58LlXaqU_u'), ('TECLADO_Y_MOUSE_ROSA_ MAGEGGEE_GK710', '17Q0_FwqpVvPN-xaH0Ud0wdlBItvG8mLr'), ('3DS', '13IXfP-JSSjwpOhr51bNaxdk4CaJhwiKJ'), ('RAZER_KRAKEN_KITTY', '1reT_o2GguU4Rwr4ZJ5IiEubJz26OrtHz'), ('CUADRO_DE_JUEGOS_DE_FROMSOFTWARE', '1LoBPvhsRRKtJJqP6DEbnpICEvucZVLAi'), ('ASUS_ZENPAD_10', '1DnPYwlKNuTd4GrQ4rnaukwvJgSV67k3I'), ('TECLADO_PIANO_YAMAHA', '1KjLWCXpnyPalreuGPcf9ttCcCO559DK2'), ('CUADRO_CANVAS__DE_MAJORAS_MASK', '1zmKzNQRrFEhTGTWXL75aWfQNUx6232iK'), ('BICI_FIJA_', '1wHpQb43UDGrFp51r_4ph6mU879M-rGbo'), ('WII', '1AoCTRRjrY16X5af5IzQ1d5Y_BcR96JHf'), ('RASPBERRY_PI_4_B_4GB_RAM', '1K6_Rf4oTiMn12m782rY5dR9t2COmp8Od'), ('CARGADOR_PORTATIL_APC_M10', '1-8q1VVyEv8TUxalRUO9wo7fR8oH4KGHY'), ('LENTE_CAMARA_10-18MM', '16o64rvttQ4SiZ36Mu8_huwBcZ9nocVmt'), ('SILLON_GRIS_3_CUER

In [23]:
#Voy a asociar los productos con las carpetas e ids correspondientes

def encontrar_carpeta_similar(nombre_producto, carpetas):
    # Extrae solo los nombres de las carpetas para la comparación
    nombres_carpetas = [carpeta[0] for carpeta in carpetas]
    
    # Encuentra el nombre de la carpeta más similar
    nombre_carpeta_similar, _ = process.extractOne(nombre_producto, nombres_carpetas)
    
    # Encuentra la tupla completa (nombre e ID) para la carpeta coincidente
    carpeta_coincidente = next((carpeta for carpeta in carpetas if carpeta[0] == nombre_carpeta_similar), None)
    
    return carpeta_coincidente

# Ahora, itera sobre los nombres de los productos en df2 y encuentra las carpetas correspondientes
resultados = []
for nombre_producto in df2['Producto']:
    carpeta_similar = encontrar_carpeta_similar(nombre_producto, carpetas)
    if carpeta_similar:
        # Guarda el resultado como una tupla de (nombre_producto, nombre_carpeta, id_carpeta)
        resultados.append((nombre_producto, carpeta_similar[0], carpeta_similar[1]))

# resultados contiene una lista de productos junto con el nombre e ID de la carpeta más similar en Drive
print(resultados)

[('Auriculares Razer Kraken Kitty', 'RAZER_KRAKEN_KITTY', '1reT_o2GguU4Rwr4ZJ5IiEubJz26OrtHz'), ('Batidora Mondial', 'BATIDORA_MONDIAL', '1iyIVWn9xKFSsKe8rQ-F1Ep7KSsB-JY2t'), ('Calentador de agua ', 'CALENTADOR_DE_AGUA_', '1KLC2sPBFvuMr6lzRoNn8ja4C1-PmvuOt'), ('Camara profesional Canon 80D', 'CAMARA_PROFESIONAL_CANON_80D', '18_zCfz5gQADgjH3Qsz9qsJM-H4rC8XEf'), ('Lente camara canon 10-18mm', 'LENTE_CAMARA_10-18MM', '16o64rvttQ4SiZ36Mu8_huwBcZ9nocVmt'), ('Cargador portatil APC M10', 'CARGADOR_PORTATIL_APC_M10', '1-8q1VVyEv8TUxalRUO9wo7fR8oH4KGHY'), ('Casio fx-95ES PLUS', 'CASIO_FX-95ES_PLUS', '1hnjdXml6iwAKXpOGDT-_McSthXzqiC8N'), ('Casio fx-991ES', 'CASIO_FX-991ES', '1_PSi-NaAAvZO2v6xDaDDjBSSkNSTJiv0'), ('Cuadro de juegos de fromsoftware', 'CUADRO_DE_JUEGOS_DE_FROMSOFTWARE', '1LoBPvhsRRKtJJqP6DEbnpICEvucZVLAi'), ('Cuadro  de majoras mask', 'CUADRO_CANVAS__DE_MAJORAS_MASK', '1zmKzNQRrFEhTGTWXL75aWfQNUx6232iK'), ('Deku', 'DEKU', '1YbgA4wmUveF3Aa25LmknFYJHkD-dHUjB'), ('Futón japonés', 'FUTO

In [24]:
#Ahora voy a descargar las imagenes localmente
def descargar_imagenes(service, resultados):
    os.makedirs('imagenes_descargadas', exist_ok=True)
    
    for producto, _, carpeta_id in resultados:
        query = f"'{carpeta_id}' in parents and mimeType contains 'image/' and trashed=false"
        response = service.files().list(q=query, fields="files(id, name)").execute()
        items = response.get('files', [])
        
        if not items:
            print(f"No se encontraron imágenes en la carpeta: {producto}")
        else:
            for i, item in enumerate(items):
                file_id = item['id']
                original_file_name = item['name']
                file_extension = os.path.splitext(original_file_name)[1]  # Extrae la extensión del archivo original
                normalized_product_name = producto.replace(' ', '_').replace('/', '_').replace('.', '_').replace('\'' , '_').replace('\"' , '_')
                
                # Construye el nuevo nombre del archivo manteniendo la extensión original
                new_file_name = f"{normalized_product_name}_{i+1}{file_extension}"
                file_path = os.path.join('imagenes_descargadas', new_file_name)
                
                if os.path.exists(file_path):
                    print(f"El archivo ya existe: {file_path}")
                    continue
                
                request = service.files().get_media(fileId=file_id)
                fh = io.FileIO(file_path, 'wb')
                downloader = MediaIoBaseDownload(fh, request)
                
                done = False
                while not done:
                    _, done = downloader.next_chunk()
                
                print(f"Descargada '{file_path}'")

descargar_imagenes(service, resultados)

El archivo ya existe: imagenes_descargadas/Auriculares_Razer_Kraken_Kitty_1.HEIC
El archivo ya existe: imagenes_descargadas/Batidora_Mondial_1.HEIC
El archivo ya existe: imagenes_descargadas/Calentador_de_agua__1.HEIC
El archivo ya existe: imagenes_descargadas/Camara_profesional_Canon_80D_1.HEIC
El archivo ya existe: imagenes_descargadas/Lente_camara_canon_10-18mm_1.HEIC
El archivo ya existe: imagenes_descargadas/Cargador_portatil_APC_M10_1.HEIC
El archivo ya existe: imagenes_descargadas/Cargador_portatil_APC_M10_2.HEIC
El archivo ya existe: imagenes_descargadas/Casio_fx-95ES_PLUS_1.HEIC
El archivo ya existe: imagenes_descargadas/Casio_fx-991ES_1.HEIC
El archivo ya existe: imagenes_descargadas/Cuadro_de_juegos_de_fromsoftware_1.HEIC
El archivo ya existe: imagenes_descargadas/Cuadro__de_majoras_mask_1.HEIC
El archivo ya existe: imagenes_descargadas/Deku_1.HEIC
El archivo ya existe: imagenes_descargadas/Futón_japonés_1.HEIC
El archivo ya existe: imagenes_descargadas/Futón_japonés_2.HEIC


In [25]:
from PIL import Image as PILImage
from pillow_heif import register_heif_opener
register_heif_opener()

directorio_imagenes = 'imagenes_descargadas' 
directorio_salida = 'imagenes_convertidas'

# Crea el directorio de salida si no existe
os.makedirs(directorio_salida, exist_ok=True)

# Lista todos los archivos en el directorio de imágenes
for archivo in os.listdir(directorio_imagenes):
    # Verifica si el archivo es una imagen HEIC
    if archivo.endswith(".HEIC") or archivo.endswith(".heic"):
        ruta_completa_heic = os.path.join(directorio_imagenes, archivo)
        nombre_base = os.path.splitext(archivo)[0]
        ruta_completa_jpg = os.path.join(directorio_salida, nombre_base + ".jpg")
        
        # Abre y convierte la imagen
        with PILImage.open(ruta_completa_heic) as img:
            img.convert("RGB").save(ruta_completa_jpg, "JPEG")
            print(f"Convertido {archivo} a JPG.")

Convertido Lente_camara_canon_10-18mm_1.HEIC a JPG.
Convertido Reposeras_Just_Home_Collection_Sao_Paulo___1.HEIC a JPG.
Convertido Mesa_escritorio_eléctrica_regulable_hafele_2_motores_125kg_1.heic a JPG.
Convertido Casio_fx-95ES_PLUS_1.HEIC a JPG.
Convertido Camara_profesional_Canon_80D_1.HEIC a JPG.
Convertido Soporte_Gadnic_monitor_1.HEIC a JPG.
Convertido Tacho_gigante_1.HEIC a JPG.
Convertido Silla_Erasmo_Fox_Slider_2.HEIC a JPG.
Convertido Monopatin_Electrico_Logus_L7pro_700watts_2.HEIC a JPG.
Convertido Teclado_Y_Mouse_Magegee_Gk710_Color_Rosa_2.HEIC a JPG.
Convertido Batidora_Mondial_1.HEIC a JPG.
Convertido Google_nest_2.HEIC a JPG.
Convertido PS_Vita_hackeada__1.HEIC a JPG.
Convertido TEG_1.HEIC a JPG.
Convertido Rollers_Bladerunner_Pro_80_1.heic a JPG.
Convertido Televisión_Samsung_4K_1.heic a JPG.
Convertido Deku_1.HEIC a JPG.
Convertido Cargador_portatil_APC_M10_2.HEIC a JPG.
Convertido Auriculares_Razer_Kraken_Kitty_1.HEIC a JPG.
Convertido Google_nest_3.HEIC a JPG.
Conver

In [26]:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, KeepTogether
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from PIL import Image as PILImage
from PIL import ImageFont
from PIL import ImageDraw
import os
from reportlab.lib.enums import TA_JUSTIFY

documento = SimpleDocTemplate("Venta_Productos.pdf", pagesize=letter)
flowables = []

styles = getSampleStyleSheet()
style_normal = styles['Normal']
style_normal.fontSize = 12
style_normal.alignment = TA_JUSTIFY  # Ajusta el texto para que esté justificado

# Añade el texto introductorio
texto_introductorio = """
<font color="green">
El precio que figura está en <b>dólares</b>.<br />
Se puede pagar en <b>pesos</b> al <b>valor venta</b> en <b>dolarhoy.com</b>.<br /> 
Se acepta <b>transferencia/efectivo</b>. <br />Whatsapp de contacto: <b>11-3497-6502</b>.<br />
Se retira por <b>Nordelta</b>.</font>
"""
ps = ParagraphStyle('title', fontSize=16, leading=20)
parrafo_introductorio = Paragraph(texto_introductorio, ps)
flowables.append(parrafo_introductorio)
flowables.append(Spacer(1, 2 * inch))
df2=df2.sort_values(by='Interes')
for index, fila in df2.iterrows():
    elementos_producto = []  # Esta lista almacenará temporalmente los elementos de cada producto
    
    # Información del producto
    texto_producto = f"<b>{fila['Producto']}</b> - <b><font color=\"red\">${fila['Precio Venta (USD)']}</font></b><br/>{fila['Observaciones']}"
    parrafo_producto = Paragraph(texto_producto, style_normal)
    elementos_producto.append(parrafo_producto)
    elementos_producto.append(Spacer(1, 0.2 * inch))
    
    # Lista para almacenar las imágenes del producto
    imagenes_producto = []
    
    nombre_producto_normalizado = fila["Producto"].replace(' ', '_').replace('/', '_').replace('.', '_').replace('\'' , '_').replace('\"' , '_')
    for n in range(1, 5):  # Ajusta según el número de imágenes por producto
        nombre_imagen = f'imagenes_convertidas/{nombre_producto_normalizado}_{n}.jpg'
        if not os.path.exists(nombre_imagen):
            continue
        try:
            with PILImage.open(nombre_imagen) as pil_img:
                ancho, alto = pil_img.size
                if ancho > alto:
                    img = Image(nombre_imagen, width=2.2*inch, height=1.7*inch)
                else:
                    img = Image(nombre_imagen, width=1.7*inch, height=2.2*inch)

      
                #end images creation
            imagenes_producto.append(img)
        except Exception as e:
            print("Exception cafetera")
            print(e)
            pass
    
    # Si hay imágenes, crea una tabla para ellas y agrégala a los elementos del producto
    if imagenes_producto:
        tabla_imagenes = Table([imagenes_producto], colWidths=[2*inch for img in imagenes_producto], hAlign='CENTER')
        elementos_producto.append(tabla_imagenes)
        elementos_producto.append(Spacer(1, 0.2 * inch))
    
    # Usa KeepTogether en los elementos del producto
    producto_completo = KeepTogether(elementos_producto)
    flowables.append(producto_completo)

documento.build(flowables)

In [27]:


from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, KeepTogether

from PIL import Image as PILImage
from PIL import ImageFont
from PIL import ImageDraw
import os


df2=df2.sort_values(by='Interes')
os.makedirs('imagenes_con_precio', exist_ok=True)
directorio_salida = 'imagenes_con_precio'
for index, fila in df2.iterrows():


    nombre_producto_normalizado = fila["Producto"].replace(' ', '_').replace('/', '_').replace('.', '_').replace('\'' , '_').replace('\"' , '_')
    for n in range(1, 5):  # Ajusta según el número de imágenes por producto
        nombre_imagen = f'imagenes_convertidas/{nombre_producto_normalizado}_{n}.jpg'
        if not os.path.exists(nombre_imagen):
            continue
        try:
            img = PILImage.open(nombre_imagen)

            #create images as well with price on them
            draw = ImageDraw.Draw(img)
            # font = ImageFont.truetype(<font-file>, <font-size>)
            font_title = ImageFont.truetype(r'\\wsl.localhost\UbuntuNew\home\mtejada\projects\vendemostodo\ARIAL.TTF', 150)
            font_precio = ImageFont.truetype(r'\\wsl.localhost\UbuntuNew\home\mtejada\projects\vendemostodo\ARIAL.TTF', 150)
            # draw.text((x, y),"Sample Text",(r,g,b))
            draw.text((5, 5),fila['Producto'],font=font_title,fill=(255,0,0))
            precio=fila['Precio Venta (USD)']*1220
            precio=int(precio)
            draw.text((5, 180),"$"+f"{precio:,}".replace(',','.')+" pesos",font=font_precio,fill=(255,0,0))
            draw.text((5, 320),"Whatsapp 11-3497-6502",font=font_precio,fill=(255,0,0))
            ruta_completa_con_precio = os.path.join(directorio_salida,f'{nombre_producto_normalizado}_{n}.jpg')
            img.save(ruta_completa_con_precio, optimize=True, quality=50)
            
            #img.show()
        except Exception as e:
            print("Exception cafetera")
            print(e)
            pass


Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
Exception cafetera
cannot open resource
